* @Author: Sankar
* @Date: 2021-05-28 12:51:25
* @Last Modified by: Sankar
* @Last Modified time: 2021-05-30 15:38:09
* @Title : Twitter Sentiment Analysis

In [0]:
import asyncio 
from azure.eventhub.aio import EventHubProducerClient 
from azure.eventhub import EventData 
from tweepy import Stream 
from tweepy import OAuthHandler 
from tweepy.streaming import StreamListener 
import json 
import nltk 
from textblob import TextBlob, Word, Blobber 
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger') 
nltk.download('brown')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data] /root/nltk_data...
[nltk_data] Package averaged_perceptron_tagger is already up-to-
[nltk_data] date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data] Package brown is already up-to-date!
Out[1]: True

In [0]:
def getBlobSentiment(text): 
  blob = TextBlob(text)
  i=0.00
  for sentence in blob.sentences:
    i=i+sentence.sentiment.polarity
#     print(sentence.sentiment.polarity)
  if i >0: 
    return 1
  elif i<0:
    return -1
  else:
    return 0

In [0]:
conn_string = <Connection_String>
eventhub_name = <Event_Hub_name>
async def run(text):
  producer = EventHubProducerClient.from_connection_string(conn_str = conn_string, eventhub_name = eventhub_name)
  async with producer:# Create a batch.
    event_data_batch = await producer.create_batch()
    event_data_batch.add(EventData(text))
    await producer.send_batch(event_data_batch)  

In [0]:
class listener(StreamListener): 
  
  def on_data(self, data): 
    loop = asyncio.get_event_loop()

    t=json.loads(data)
            
    d1 = {"text":t["text"] ,"sentiment": getBlobSentiment(t["text"]) ,"created_at" :t["created_at"]  }
    print (type(data))     
    loop.run_until_complete(run(json.dumps(d1)))
    print(d1)
    
    return(True)
   
  def on_error(self, status):
    print(status)


In [0]:
consumer_key = config('CONSUMER_KEY')
consumer_secret = config('CONSUMER_SECRET')
access_token = config('ACCESS_TOKEN')
access_secret = config('ACCESS_SECRET')

auth = OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_secret)

twitterStream = Stream(auth, listener()) 
twitterStream.filter(track=["trump"])

<class 'str'>
VendorLinkDetach('com.microsoft:argument-out-of-range: The specified partition is invalid for an EventHub partition sender or receiver. It should be between 0 and 0.\r\nParameter name: PartitionId TrackingId:6bc6d9b375444158b74adde5a98fe38e_G11S2, SystemTracker:gateway5, Timestamp:2021-05-29T04:12:33')
VendorLinkDetach('com.microsoft:argument-out-of-range: The specified partition is invalid for an EventHub partition sender or receiver. It should be between 0 and 0.\r\nParameter name: PartitionId TrackingId:2f8e2161ff974db09d520d0d10df0dcd_G5S2, SystemTracker:gateway5, Timestamp:2021-05-29T04:12:37')
VendorLinkDetach('com.microsoft:argument-out-of-range: The specified partition is invalid for an EventHub partition sender or receiver. It should be between 0 and 0.\r\nParameter name: PartitionId TrackingId:890fd9eb56af4fd188437880f80f2511_G2S2, SystemTracker:gateway5, Timestamp:2021-05-29T04:12:41')
VendorLinkDetach('com.microsoft:argument-out-of-range: The specified partition is invalid for an EventHub partition sender or receiver. It should be between 0 and 0.\r\nParameter name: PartitionId TrackingId:890fd9eb56af4fd188437880f80f2511_G2S2, SystemTracker:gateway5, Timestamp:2021-05-29T04:12:48')

--------------------------------------------------------------------------- 
 ClientClosedError Traceback (most recent call last)
 /databricks/python/lib/python3.8/site-packages/azure/eventhub/aio/_producer_client_async.py in send_batch (self, event_data_batch, timeout, **kwargs) 
 306 try : 
 --> 307 await cast(EventHubProducer, self._producers[partition_id]).send(
 308 to_send_batch , timeout = timeout

 /databricks/python/lib/python3.8/site-packages/azure/eventhub/aio/_producer_async.py in send (self, event_data, partition_key, timeout) 
 237 with send_context_manager ( ) as child : 
 --> 238 self . _check_closed ( ) 
 239 wrapper_event_data = self . _wrap_eventdata ( event_data , child , partition_key ) 

 /databricks/python/lib/python3.8/site-packages/azure/eventhub/aio/_client_base_async.py in _check_closed (self) 
 418 if self . closed : 
 --> 419 raise ClientClosedError(
 420 "{} has been closed. Please create a new one to handle event data.".format(

 ClientClosedError : EHProducer-cd913a0b-b003-4f34-b8de-427d336e1d0c has been closed. Please create a new one to handle event data.

During handling of the above exception, another exception occurred:

 VendorLinkDetach Traceback (most recent call last)
 /databricks/python/lib/python3.8/site-packages/azure/eventhub/aio/_client_base_async.py in _do_retryable_operation (self, operation, timeout, **kwargs) 
 481 if operation_need_param : 
 --> 482 return await operation(
 483 timeout_time = timeout_time , 

 /databricks/python/lib/python3.8/site-packages/azure/eventhub/aio/_producer_async.py in _send_event_data (self, timeout_time, last_exception) 
 142 if self . _unsent_events : 
 --> 143 await self . _open ( ) 
 144 self . _set_msg_timeout ( timeout_time , last_exception ) 

 /databricks/python/lib/python3.8/site-packages/azure/eventhub/aio/_client_base_async.py in _open (self) 
 440 )
 --> 441 while not await self . _handler . client_ready_async ( ) : 
 442 await asyncio . sleep ( 0.05 , loop = self . _loop ) 

 /databricks/python/lib/python3.8/site-packages/uamqp/async_ops/client_async.py in client_ready_async (self) 
 361 return False 
 --> 362 if not await self . _client_ready_async ( ) : 
 363 await self . _connection . work_async ( ) 

 /databricks/python/lib/python3.8/site-packages/uamqp/async_ops/client_async.py in _client_ready_async (self) 
 518 return False 
 --> 519 if self . message_handler . get_state ( ) == constants . MessageSenderState . Error : 
 520 raise errors.MessageHandlerError(

 /databricks/python/lib/python3.8/site-packages/uamqp/sender.py in get_state (self) 
 191 try : 
 --> 192 raise self . _error
 193 except TypeError : 

 VendorLinkDetach : com.microsoft:argument-out-of-range: The specified partition is invalid for an EventHub partition sender or receiver. It should be between 0 and 0.
Parameter name: PartitionId TrackingId:890fd9eb56af4fd188437880f80f2511_G2S2, SystemTracker:gateway5, Timestamp:2021-05-29T04:12:48

During handling of the above exception, another exception occurred:

 ConnectError Traceback (most recent call last)
 <command-603001068614098> in <module> 
 8 
 9 twitterStream = Stream ( auth , listener ( ) ) 
 ---> 10 twitterStream . filter ( track = [ "trump" ] ) 

 /databricks/python/lib/python3.8/site-packages/tweepy/streaming.py in filter (self, follow, track, is_async, locations, stall_warnings, languages, encoding, filter_level) 
 472 self . body [ 'filter_level' ] = filter_level . encode ( encoding ) 
 473 self . session . params = { 'delimited' : 'length' } 
 --> 474 self . _start ( is_async ) 
 475 
 476 def sitestream(self, follow, stall_warnings=False,

 /databricks/python/lib/python3.8/site-packages/tweepy/streaming.py in _start (self, is_async) 
 387 self . _thread . start ( ) 
 388 else : 
 --> 389 self . _run ( ) 
 390 
 391 def on_closed ( self , resp ) : 

 /databricks/python/lib/python3.8/site-packages/tweepy/streaming.py in _run (self) 
 318 # call a handler first so that the exception can be logged. 
 319 s